# Zpracování e-mailů s využitím GROQ API pro extrakci IP adres a časových údajů

V rámci kybernetické bezpečnosti je důležité efektivně zpracovávat e-maily, které obsahují informace o potenciálních útocích nebo zneužití. Vzhledem k tomu, že e-maily mohou obsahovat různé formáty a struktury a je tedy prakticky nemožné mít automatické parsování na všechny možné varianty, využijeme některý jazykový model z GROQ API pro extrakci relevantních informací, jako jsou IP adresy a časy incidentů.

Vaším úkolem je napsat Python skript, který:
1. Načte fiktivní e-maily obsahující informace o IP adresách a časech incidentů.
2. Použije GROQ API k extrakci těchto informací.
    - vhodně nastaví prompt pro jazykový model, aby dokázal rozpoznat různé formáty časových údajů (např. "včera", "dnes ráno") a převedl je na absolutní časová razítka ve formátu ISO 8601. 
    - Zohlední čas doručení e-mailu jako kontext pro relativní časy.
    - Extrahuje infromace ve strojově zpracovatelném formátu, který dále využijete.
3 Výsledek extrakce vypíše do konzole 
4. (Bonus) Extrahované informace odešle na kontrolní API endpoint s URL https://letni-skola-api.12137-a.fsid.cvut.cz/api/v1/abuse-reports
    - výsledek musí být ve formátu JSON, kde každý objekt obsahuje:
      - `email_id`: ID e-mailu
      - `ip_address`: extrahovaná IP adresa
      - `timestamp`: přesný čas incidentu ve formátu ISO 8601
    - ukázka zprávného formátu odpovědi (v tomoto případě pro e-mail s ID 1, který obsahuje dvě IP adresy):
        ```json
        [
            {
                "email_id": 1,
                "ip_address": "1.2.3.4",
                "timestamp": "2025-08-06T10:00:00Z"
            },
            {
                "email_id": 1,
                "ip_address": "5.6.7.8",
                "timestamp": "2025-08-06T10:00:00Z"
            }
        ]
    - Odesílání dat na kontrolní API by mělo být provedeno pomocí HTTP POST požadavku.
    - Použijte Basic Auth pro autentizaci, kde použijete vaše uživatelské jméno pro letní školu a heslo je `letni-skola`
    - jako odpověď očekávejte obdobný JSON objekt kde ke každé ip adrese bude přidána hodnota `result` s hodnotou `OK` nebo `ERROR` v případě, že se nepodařilo IP adresu zpracovat.

In [2]:
import requests
import json
from groq import Groq

# Konfigurace GROQ API
GROQ_API_KEY = ""
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# Bonus údaje pro přístup ke kontrolnímu API
USERNAME="letni-skola-uzivatel"
PASSWORD="letni-skola-heslo"

# Seznam e-mailů
emails = [
  {
    "id": 0,
    "received_at": "2025-08-06T10:00:00Z",
    "text": "Dobrý den, z IP adresy 185.122.204.15 probíhal včera večer (kolem 20:30) sken portů. Byla zjištěna i aktivita z 209.85.232.189, a to přibližně ve stejnou dobu."
  },
  {
    "id": 1,
    "received_at": "2025-08-06T12:00:00Z",
    "text": "CHCI NAHLÁSIT SPAM Z 64.233.165.101!! DĚJE SE TO UŽ PÁR HODIN, DNES RÁNO, TAK KOLEM 9:15. UŽ MĚ TO VÁŽNĚ ŠTVE! Děkuji. Marie."
  },
  {
    "id": 2,
    "received_at": "2025-08-06T11:00:00Z",
    "text": "Dear Abuse Department, we have detected a brute-force attack from 172.217.16.142 on our SSH server at 2025-08-06T10:35:00Z. Please investigate."
  },
  {
    "id": 3,
    "received_at": "2025-08-06T15:30:00Z",
    "text": "Dobrý den, před chvílí mi volal počítačový guru a říkal, že mám napsat na abuse, že se mi někdo nabourává do počítače. Prý je to ta adresa 132.266.158.32. Nic víc nevím. Děkuji. Petr Novák"
  },
  {
    "id": 4,
    "received_at": "2025-08-06T20:00:00Z",
    "text": "AUTOMATED ALERT: High-volume traffic detected from IP 198.51.100.1 and 203.0.113.10. Timestamp: 2025-08-06T19:55:00Z. Severity: High."
  },
  {
    "id": 5,
    "received_at": "2025-08-06T17:00:00Z",
    "text": "Dobrý den, posílám ten IP adresa útok 198.19.24.1, předevčírem v poledne. To mi říkal soused. Mám to poslat."
  },
  {
    "id": 6,
    "received_at": "2025-08-06T14:45:00Z",
    "text": "Zdravím, dneska odpoledne jsem si všimnul podezřelých pokusů o přihlášení. Bylo to z adresy 130.208.5.1 v 14:00."
  },
  {
    "id": 7,
    "received_at": "2025-08-06T19:00:00Z",
    "text": "Ahoj, chci se zeptat, jak se mám přihlásit na Wi-Fi na kolejích? Ten login mi nefunguje."
  },
  {
    "id": 8,
    "received_at": "2025-08-06T21:30:00Z",
    "text": "AUTOMATED ALERT: Honeypot SSH login attempts log. The following IP addresses have been blocked for suspicious activity. The events occurred in the listed order between 2025-08-06T21:20:15Z and 2025-08-06T21:25:45Z, with roughly 30-second intervals between them.\n\n - 198.51.100.12\n - 203.0.113.55\n - 172.16.2.20\n - 10.10.10.2\n - 192.0.2.11\n - 198.51.100.7\n - 203.0.113.88\n - 172.16.2.33\n - 10.10.10.25\n - 192.0.2.15\n\nPlease review and take necessary action."
  }
]